**대규모 텐서플로 모델 훈련과 배포**

# 설정

In [1]:
from tensorflow import config

config.list_physical_devices('GPU')

2025-08-15 05:56:40.044199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755237400.208690      32 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755237400.257632      32 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755237400.742072      32 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755237400.742113      32 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755237400.742115      32 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 텐서플로 모델 서빙하기

먼저 TF 서빙을 사용하여 모델을 배포한 다음 Google Vertex AI에 배포해 본다.

## 텐서플로 서빙 사용

가장 먼저 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보낸다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할한다.

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend, layers
from tensorflow import keras
import tensorflow as tf
from pathlib import Path

mnist_ = mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist_
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
backend.clear_session()
model = keras.Sequential()
model.add(keras.Input((28, 28), dtype=tf.uint8))
model.add(layers.Flatten())
model.add(layers.Rescaling(1 / 255))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
model_name = 'my_mnist_model'
model_version = '0001'
model_path = Path(model_name) / model_version
model.export(model_path)

I0000 00:00:1755237405.484436      32 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5557 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:06:00.0, compute capability: 8.6


Epoch 1/10


I0000 00:00:1755237406.915204     115 service.cc:152] XLA service 0x750d500071b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755237406.915241     115 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-08-15 05:56:46.953952: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1755237407.023331     115 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-15 05:56:47.737483: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36_0', 92 bytes spill stores, 96 bytes spill loads

2025-08-15 05:56:47.839488: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36

  81/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2437 - loss: 2.1545

I0000 00:00:1755237410.145837     115 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7191 - loss: 1.0699

2025-08-15 05:56:57.552324: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 296 bytes spill stores, 296 bytes spill loads

2025-08-15 05:56:57.640956: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-15 05:56:57.708274: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-15 05:56:57.848613: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 792 bytes spill stores, 844 bytes spill loads

2025-08-15 05:56:57.968870: I external/local_xla

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.7191 - loss: 1.0697 - val_accuracy: 0.8994 - val_loss: 0.3720
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8985 - loss: 0.3696 - val_accuracy: 0.9182 - val_loss: 0.3030
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9120 - loss: 0.3136 - val_accuracy: 0.9260 - val_loss: 0.2719
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9217 - loss: 0.2793 - val_accuracy: 0.9330 - val_loss: 0.2444
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9302 - loss: 0.2501 - val_accuracy: 0.9372 - val_loss: 0.2249
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9341 - loss: 0.2326 - val_accuracy: 0.9434 - val_loss: 0.2100
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9400 - loss: 0.2162 - val_accuracy: 0.9470 - val_loss: 0.1965
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9413 - loss: 0.2022 - val_accura

INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Saved artifact at 'my_mnist_model/0001'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  128703894991696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703894992848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703894990160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703894993424: TensorSpec(shape=(), dtype=tf.resource, name=None)


파일 트리를 살핀다:

In [3]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사한다:

In [4]:
!saved_model_cli show --dir '{model_path}'

2025-08-15 05:57:33.833471: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755237453.847866     928 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755237453.853773     928 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755237453.869345     928 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755237453.869408     928 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755237453.869414     928 computation_placer.cc:177] computation placer alr

In [5]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2025-08-15 05:57:36.717421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755237456.730769     945 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755237456.734890     945 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755237456.746539     945 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755237456.746575     945 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755237456.746581     945 computation_placer.cc:177] computation placer alr

In [6]:
!saved_model_cli show --dir '{model_path}' --tag_set serve --signature_def serving_default

2025-08-15 05:57:39.281150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755237459.294448     961 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755237459.298531     961 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755237459.310026     961 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755237459.310061     961 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755237459.310067     961 computation_placer.cc:177] computation placer alr

더 자세한 내용을 보려면 다음 명령을 실행한다:
```ipython
!saved_model_cli show --dir '{model_path}' --all
```

### 텐서플로 서빙 설치 및 시작하기

도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행한다.
```bash
docker pull tensorflow/serving:latest-gpu

docker run -e MODEL_NAME=my_mnist_model --gpus all -it -p 8500:8500 -p 8501:8501 -v path\to\my_mnist_model:/models/my_mnist_model tensorflow/serving:latest-gpu
```

### REST API로 TF 서빙에 쿼리하기

다음으로 TF 서빙에 REST 쿼리를 전송한다:

In [7]:
import json

X_new = X_test[:3]  # 분류할 새로운 숫자 이미지가 3개 있다고 가정한다.
request_json = json.dumps({'signature_name': 'serving_default', 'instances': X_new.tolist()})

In [8]:
request_json[:100] + '...' + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

이제 텐서플로 서빙의 REST API를 사용하여 예측한다:

In [9]:
import requests

server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(server_url, request_json)
response.raise_for_status()  # 오류 발생 시 예외 발생
response = response.json()

In [10]:
import numpy as np

y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

### gRPC API로 TF 서빙에 쿼리하기

In [11]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = 'serving_default'
input_name = 'keras_tensor'
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

In [12]:
request

model_spec {
  name: "my_mnist_model"
  signature_name: "serving_default"
}
inputs {
  key: "keras_tensor"
  value {
    dtype: DT_UINT8
    tensor_shape {
      dim {
        size: 3
      }
      dim {
        size: 28
      }
      dim {
        size: 28
      }
    }
    tensor_content: "\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\00

In [13]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel('172.17.0.3:8500')
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

응답을 텐서로 변환한다:

In [14]:
output_name = 'output_0'
outputs_proto = response.outputs[output_name]
y_proba = tf.make_ndarray(outputs_proto)

In [15]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]],
      dtype=float32)

### 새 모델 버전 배포

In [16]:
# 새로운 MNIST 모델 버전 빌드 및 훈련
model = keras.Sequential()
model.add(keras.Input((28, 28), dtype=tf.uint8))
model.add(layers.Flatten())
model.add(layers.Rescaling(1 / 255))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10


2025-08-15 05:57:44.866586: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-08-15 05:57:45.226363: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 944 bytes spill stores, 944 bytes spill loads

2025-08-15 05:57:45.229691: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 940 bytes spill stores, 940 bytes spill loads

2025-08-15 05:57:45.368878: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 1180 bytes spill stores, 1240 bytes spill loads

2025-08-15 05:57:45.836627: I external

1700/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6714 - loss: 1.1561

2025-08-15 05:57:51.148118: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 272 bytes spill stores, 272 bytes spill loads

2025-08-15 05:57:51.485640: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 944 bytes spill stores, 944 bytes spill loads

2025-08-15 05:57:51.560059: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 988 bytes spill stores, 988 bytes spill loads

2025-08-15 05:57:51.681476: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 956 bytes spill stores, 956 bytes spill loads

2025-08-15 05:57:52.109039: I external/l

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6729 - loss: 1.1509

2025-08-15 05:57:54.901311: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads

2025-08-15 05:57:54.901518: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 348 bytes spill stores, 348 bytes spill loads

2025-08-15 05:57:55.049362: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads

2025-08-15 05:57:55.125166: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 816 bytes spill stores, 816 bytes spill loads



1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.6730 - loss: 1.1507 - val_accuracy: 0.9072 - val_loss: 0.3364
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9030 - loss: 0.3348 - val_accuracy: 0.9248 - val_loss: 0.2685
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9213 - loss: 0.2728 - val_accuracy: 0.9310 - val_loss: 0.2330
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9296 - loss: 0.2418 - val_accuracy: 0.9362 - val_loss: 0.2124
Epoch 5/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9353 - loss: 0.2211 - val_accuracy: 0.9446 - val_loss: 0.1935
Epoch 6/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9436 - loss: 0.1944 - val_accuracy: 0.9496 - val_loss: 0.1773
Epoch 7/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9458 - loss: 0.1876 - val_accuracy: 0.9552 - val_loss: 0.1644
Epoch 8/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9519 - loss: 0.1691 - val_accura

In [17]:
model_version = '0002'
model_path = Path(model_name) / model_version
model.export(model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


Saved artifact at 'my_mnist_model/0002'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  128703851135056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703851136976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703894984016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703851141008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703851140624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703851140432: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [18]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

**경고**: 텐서플로 서빙이 새 모델을 로드하기까지 잠시 기다려야 할 수 있다.

In [19]:
server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = requests.post(server_url, request_json)
response.raise_for_status()
response = response.json()

In [20]:
response.keys()

dict_keys(['predictions'])

In [21]:
y_proba = np.array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.97, 0.01, 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.98, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]])

# 모바일 또는 임베디드 디바이스에 모델 배포하기

In [22]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open('my_converted_savedmodel.tflite', 'wb') as f:
    f.write(tflite_model)

W0000 00:00:1755237514.644716      32 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1755237514.644756      32 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-15 05:58:34.648421: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: my_mnist_model/0002
2025-08-15 05:58:34.651553: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-15 05:58:34.651566: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: my_mnist_model/0002
I0000 00:00:1755237514.655503      32 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-08-15 05:58:34.656130: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-15 05:58:34.679068: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: my_mnist_model/0002
2025-08-15 05:58:34.684371: I tensorflow/cc/saved_model/loader.cc:471] Sa

In [23]:
# 케라스 모델을 변환한다.
converter = lite.TFLiteConverter.from_keras_model(model)

In [24]:
converter.optimizations = [lite.Optimize.DEFAULT]

In [25]:
tflite_model = converter.convert()
with open('my_converted_keras_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmph9_q7rog/assets


INFO:tensorflow:Assets written to: /tmp/tmph9_q7rog/assets


Saved artifact at '/tmp/tmph9_q7rog'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_11')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  128703851135056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703851136976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703894984016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703851141008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703851140624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  128703851140432: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1755237514.935416      32 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1755237514.935453      32 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-15 05:58:34.935573: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmph9_q7rog
2025-08-15 05:58:34.935892: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-15 05:58:34.935897: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmph9_q7rog
2025-08-15 05:58:34.938371: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-15 05:58:34.952904: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmph9_q7rog
2025-08-15 05:58:34.957556: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 21986 microseconds.


# GPU를 사용하여 계산 속도 향상하기

텐서플로우가 GPU를 볼 수 있는지 확인한다:

In [26]:
physical_gpus = config.list_physical_devices('GPU')
physical_gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

텐서플로 스크립트에서 GPU \#0과 \#1(PCI 순서 기준)만 사용하려면 스크립트를 시작하기 전에 환경 변수 `CUDA_DEVICE_ORDER=PCI_BUS_ID`와 `CUDA_VISIBLE_DEVICES=0,1`을 설정한다. 또는 스크립트 자체에서 텐서플로를 사용하기 전에 지정한다.

## GPU RAM 관리

RAM 용량을 GPU당 2GB로 제한하려면:

In [27]:
# for gpu in physical_gpus:
#     config.set_logical_device_configuration(gpu, [config.LogicalDeviceConfiguration(2048)])

점진적으로 텐서플로가 메모리를 점유하도록 하려면(프로세스가 종료될 때만 메모리를 해제한다):

In [28]:
# from tensorflow.config import experimental

# for gpu in physical_gpus:
#     experimental.set_memory_growth(gpu, True)

이와 동일하게, 텐서플로를 사용하기 전에 `TF_FORCE_GPU_ALLOW_GROWTH` 환경 변수를 `true`로 설정할 수 있다.

물리적 GPU를 두 개의 논리적 GPU로 분할한다:

In [30]:
# config.set_logical_device_configuration(
#     physical_gpus[0], [config.LogicalDeviceConfiguration(2048), config.LogicalDeviceConfiguration(2048)]
# )

In [41]:
logical_gpus = config.list_logical_devices('GPU')
logical_gpus

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

## 디바이스에 연산 및 변수 배치하기

모든 변수 및 연산 배치를 기록하려면(이 작업은 텐서플로를 임포팅한 직후에 실행해야 한다):

In [42]:
from tensorflow import debugging

tf.get_logger().setLevel("DEBUG")  # 디폴트 로그 수준은 INFO다.
debugging.set_log_device_placement(True)

In [43]:
a = tf.Variable([1., 2., 3.])  # float32 변수는 GPU로 이동한다.
a.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [44]:
b = tf.Variable([1, 2, 3])  # int32 변수는 CPU로 이동한다.
b.device

'/job:localhost/replica:0/task:0/device:CPU:0'

`tf.device()` 컨텍스트를 사용하여 원하는 장치에 변수 및 연산을 수동으로 배치할 수 있다:

In [45]:
with tf.device('/cpu:0'):
    c = tf.Variable([1., 2., 3.])
c.device

'/job:localhost/replica:0/task:0/device:CPU:0'

존재하지 않거나 커널이 없는 장치를 지정하면 텐서플로는 자동으로 기본 장치를 사용한다:

In [46]:
with tf.device('/gpu:1234'):
    d = tf.Variable([1., 2., 3.])
d.device

'/job:localhost/replica:0/task:0/device:GPU:0'

텐서플로에서 존재하지 않는 장치를 사용하려고 할 때 기본 장치로 되돌아가지 않고 예외를 발생시키려면:

In [47]:
from tensorflow import errors

config.set_soft_device_placement(False)
try:
    with tf.device('/gpu:1000'):
        d = tf.Variable([1., 2., 3.])
except errors.InvalidArgumentError as ex:
    print(ex)
config.set_soft_device_placement(True)  # 소프트 배치로 돌아가기

Could not satisfy device specification '/job:localhost/replica:0/task:0/device:GPU:1000'. enable_soft_placement=0. Supported device types [GPU, CPU]. All available devices [/job:localhost/replica:0/task:0/device:GPU:0, /job:localhost/replica:0/task:0/device:CPU:0].
